<a href="https://colab.research.google.com/github/gtbnhyujmj/BingX_Auto_Order/blob/main/BingX_Bot_Me_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install requests

In [ ]:
import time                # 引入時間模組，用來取得現在時間
import requests              # 引入requests模組，讓我們可以發送網路請求
import hmac                # 引入hmac模組，幫忙做簽名加密
from hashlib import sha256        # 引入sha256加密演算法

# 網址中間有個V3

In [ ]:
def demo():
    # 這個函式示範怎麼叫API查帳戶餘額
    path = 'https://open-api.bingx.com/openApi/swap/v3/user/balance'      # API的路徑 V3
    method = "GET"                               # 用GET方法請求資料

    # A = params["recvWindow"]
    # B = params["timestamp"]

    # params_str = "recvWindow=" + A + "&" + "timestamp=" + B   # 把字典格式的參數變成字串

    return method, path, # params_str  # 呼叫發送請求的函式，並回傳結果

method, path = demo()
demo()

('GET', 'https://open-api.bingx.com/openApi/swap/v3/user/balance')

# Params 寫在這

In [ ]:
params = {
        "recvWindow": "60000",                    # 允許的請求時間誤差，60秒（毫秒）
        "timestamp": str(int(time.time() * 1000))          # 現在時間戳記（毫秒），用來驗證請求有效
    }

# 把Params排序好，符合BX的要求

In [ ]:
def format_params(params):
    # >>> 這個函式把參數排好序，然後組成a=1&b=2這樣的字串

    keys = sorted(params.keys())  # 把參數的鍵排序

    # 建立一個空清單準備放字串
    param_list = []

    for key in keys:
        param_list.append(key + "=" + params[key]) # 這裡用簡單字串串接，把key跟value串成 "key=value" 的形式

    return "&".join(param_list) # 用 & 把所有參數連起來

final_paramas = format_params(params)
format_params(params)  ## 這個就是網頁上的 params_str = 把參數排好序，然後組成a=1&b=2這樣的字串

'recvWindow=60000&timestamp=1747285340222'

# 密鑰

In [ ]:
SECRETKEY = ""

# 建立簽證

In [ ]:
def get_signature(secret, message): # message = 上面處理好的字串  # 這裡用HMAC-SHA256演算法，根據密鑰跟參數字串做簽名
    sign = hmac.new(secret.encode(), message.encode(), sha256).hexdigest()

    # print("簽名:", sign)      # 印出簽名結果方便檢查
    return sign           # 回傳簽名字串

sign = get_signature(SECRETKEY, final_paramas)
get_signature(SECRETKEY, final_paramas)
# print("簽名:", get_signature)

'c5031cacafc9bf3b60819f6e6a126f7a4f4e5b92971eb9c8a5cb7ca30981347b'

# 組合網址

In [ ]:
# 把完整網址組合起來（字串串接方式）
url = path + "?" + final_paramas + "&signature=" + sign

# url

'https://open-api.bingx.com/openApi/swap/v3/user/balance?recvWindow=60000&timestamp=1747285340222&signature=c5031cacafc9bf3b60819f6e6a126f7a4f4e5b92971eb9c8a5cb7ca30981347b'

# 建立指定的headers

In [ ]:
# 設定HTTP請求的標頭，把APIKEY放進去

# 輸入你的公鑰
APIKEY = ""

headers = {
        'X-BX-APIKEY': APIKEY
    }

# 發送與接收

In [ ]:
# 用requests送出HTTP請求
# method = GET, 函數上面有

response = requests.request(method, url, headers=headers)
response.status_code

200

# 列印JSON

In [ ]:
response = requests.request(method, url, headers=headers)
response.text

'{"code":0,"msg":"","data":[{"userId":"1049889271413661698","asset":"USDT","balance":"0.1568","equity":"0.1568","unrealizedProfit":"0.0000","realisedProfit":"0","availableMargin":"0.1568","usedMargin":"0.0000","freezedMargin":"0.0000","shortUid":"8883392"},{"userId":"1049889271413661698","asset":"USDC","balance":"0.0003","equity":"0.0003","unrealizedProfit":"0.0000","realisedProfit":"0","availableMargin":"0.0003","usedMargin":"0.0000","freezedMargin":"0.0000","shortUid":"8883392"}]}'

# 排版JSON

In [ ]:
# 自動排版

import json  # 加在檔案開頭

# 在 print(result) 改成這樣：
parsed = json.loads(response.text)
print(json.dumps(parsed, indent=2))

{
  "code": 0,
  "msg": "",
  "data": [
    {
      "userId": "1049889271413661698",
      "asset": "USDT",
      "balance": "0.1568",
      "equity": "0.1568",
      "unrealizedProfit": "0.0000",
      "realisedProfit": "0",
      "availableMargin": "0.1568",
      "usedMargin": "0.0000",
      "freezedMargin": "0.0000",
      "shortUid": "8883392"
    },
    {
      "userId": "1049889271413661698",
      "asset": "USDC",
      "balance": "0.0003",
      "equity": "0.0003",
      "unrealizedProfit": "0.0000",
      "realisedProfit": "0",
      "availableMargin": "0.0003",
      "usedMargin": "0.0000",
      "freezedMargin": "0.0000",
      "shortUid": "8883392"
    }
  ]
}


# 紀錄

In [ ]:
# V3 catch the forever USDT Order site.
# USDT and USDC both OK

In [ ]:
# STU